practice EDA to prediction

refer to

https://www.kaggle.com/code/shreyasajal/birdclef-librosa-audio-feature-extraction/notebook
https://www.kaggle.com/code/utcarshagrawal/birdclef-audio-pytorch-tutorial

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pandas as pd
import torch
import torchaudio
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import librosa
import librosa.display
import IPython.display as ipd
import sklearn
import warnings
import seaborn as sns
from typing import List
warnings.filterwarnings('ignore')

In [ ]:
ROOTDIR = '../input/birdclef-2022'

In [ ]:
train_df=pd.read_csv('../input/birdclef-2022/train_metadata.csv')
train_df.head()

# sampling and play

In [ ]:
class AudioP():
    def __init__(self):
        self.root_dir = ROOTDIR
        # edit metadata
        self.meta = pd.read_csv(fr"{self.root_dir}/train_metadata.csv")
        self.meta["full_path"] = self.root_dir + '/' + 'train_audio' + '/' + self.meta['filename']
        
    
    def get_random_sample_by_feature(self, key: str = 'primary', value: str = None, random_state: int = 33):
        """
        get random audio links by feature
        """
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        return audio
    
    
    def play_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        print(f"{key}: {value}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)
    
    
    def play_sample_by_idx(self, idx: int = None, feature: str = 'primary_label'):
        audio = self.meta['full_path'].values[idx]
        print(f"{feature}: {self.meta[feature].values[idx]}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)
    
    
    @staticmethod
    def play_sample(file_path):
        return ipd.display(ipd.Audio(file_path)) # ipd.Audio(audio)
    
    
test_d = AudioP()
birds = list(train_df.primary_label.unique()[0:6])

## sampling

In [ ]:
audios = []
for b in birds:
    audios.append(test_d.get_random_sample_by_feature(**{'key': 'primary_label', 'value': b}))

for a in audios:
    print(a)

In [ ]:
for a, b in zip(audios, birds):
    print(f"{b}")
    test_d.play_sample(a)

In [ ]:
for b in birds:
    test_d.play_random_sample_by_feature(**{'key': 'primary_label', 'value': b})

# Loading and visualizing an audio file

- librosa.load: loads an audio file as a floating point time series and gives it's native sampling rate.
- The sampling frequency (or sample rate) is the number of samples (data points) per second in an audio.
- We can check the audio length by dividing the total number of data points by the sampling frequency.

In [ ]:
class AudioLibrosa():
    def __init__(self):
        self.root_dir = ROOTDIR
        # edit metadata
        self.meta = pd.read_csv(fr"{self.root_dir}/train_metadata.csv")
        self.meta["full_path"] = self.root_dir + '/' + 'train_audio' + '/' + self.meta['filename']
        
    
    def get_random_sample_by_feature(self, key: str = 'primary', value: str = None, random_state: int = 33):
        """
        get random audio links by feature
        """
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        return audio
        
        
    def play_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        print(f"{key}: {value}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)
    
    
    def play_sample_by_idx(self, idx: int = None, feature: str = 'primary_label'):
        audio = self.meta['full_path'].values[idx]
        print(f"{feature}: {self.meta[feature].values[idx]}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)


    @staticmethod
    def play_sample(file_path):
        return ipd.display(ipd.Audio(file_path)) # ipd.Audio(audio)
    
    
    def load_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33, trim=True):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        y, sr = librosa.load(audio)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr

    
    def load_sample_by_idx(self, idx, trim=True):
        y, sr = librosa.load(self.meta['full_path'].values[idx])
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    @staticmethod
    def load_sample(file_path, trim=True):
        y, sr = librosa.load(file_path)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
test_d = AudioLibrosa()
birds = list(train_df.primary_label.unique()[0:6])

## Trimming the leading and trailing silence

In [ ]:
trimed_samples = []
sample_rates = []
for a in audios:
    audio_file, sr = test_d.load_sample(a)
    trimed_samples.append(audio_file)
    sample_rates.append(sr)
    print('Audio File:', audio_file, '\n')
    print('Audio File shape:', np.shape(audio_file))

# Time domain features

## Waveform visualization

In [ ]:
class AudioLibrosa():
    def __init__(self):
        self.root_dir = ROOTDIR
        # edit metadata
        self.meta = pd.read_csv(fr"{self.root_dir}/train_metadata.csv")
        self.meta["full_path"] = self.root_dir + '/' + 'train_audio' + '/' + self.meta['filename']
        
    
    def get_random_sample_by_feature(self, key: str = 'primary', value: str = None, random_state: int = 33):
        """
        get random audio links by feature
        """
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        return audio
        
        
    def play_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        print(f"{key}: {value}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)
    
    
    def play_sample_by_idx(self, idx: int = None, feature: str = 'primary_label'):
        audio = self.meta['full_path'].values[idx]
        print(f"{feature}: {self.meta[feature].values[idx]}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)


    @staticmethod
    def play_sample(file_path):
        return ipd.display(ipd.Audio(file_path)) # ipd.Audio(audio)
    
    
    def load_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33, trim=True):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        y, sr = librosa.load(audio)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr

    
    def load_sample_by_idx(self, idx, trim=True):
        y, sr = librosa.load(self.meta['full_path'].values[idx])
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    @staticmethod
    def load_sample(file_path, trim=True):
        y, sr = librosa.load(file_path)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    @staticmethod
    def waveplot(samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        """
        colorlist : ["#A300F9", "#4300FF", "#009DFF", "#00FFB0", "#D9FF00", "r"]
        """
        colors = ["#009DFF", "#A300F9", "#4300FF", "#00FFB0", "#D9FF00", "r"]
        
        if not isinstance(samples, list):
            samples = [samples]
            
        if not isinstance(sample_rates, list):
            sample_rates = [sample_rates]
            
        if names and not isinstance(names, list):
            names = [names]
        
        if len(samples) == 1:
            sample = samples[0]
            sr = sample_rates[0]
            
            fig, ax = plt.subplots(1, figsize = (16, 3))
            fig.suptitle('Sound Waves', fontsize=16)
            librosa.display.waveshow(y=sample, sr=sr, color=colors[0], ax=ax)
            
            if names:
                name = names[0]
                ax.set_ylabel(name, fontsize=13)
            
        elif len(samples) > 1:
            n = len(samples)
            fig, ax = plt.subplots(n, figsize = (16, 3*n))
            fig.suptitle('Sound Waves', fontsize=16)
            
            for i in range(len(samples)):
                librosa.display.waveshow(y=samples[i], sr=sample_rates[i], color=colors[i % 6], ax=ax[i])
                
                if names:
                    ax[i].set_ylabel(names[i], fontsize=13)
        else:
            print("samples need to contain more than one")
            assert len(samples) >= 1
        
    
test_d = AudioLibrosa()
birds = list(train_df.primary_label.unique()[0:6])

In [ ]:
test_d.waveplot(trimed_samples, sample_rates, names=birds)

## Spectrogram

In [ ]:
class Config:
    n_fft=2048
    hop_length=512

In [ ]:
class AudioLibrosa():
    def __init__(self, meta: pd.DataFrame: None, n_fft: int = 1024, hop_length: int = 512):
        self.meta = meta
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.colors = ["#009DFF", "#A300F9", "#4300FF", "#00FFB0", "#D9FF00", "r"]
    
    
    # get random sample by feature
    def get_random_sample_by_feature(self, key: str = 'primary', value: str = None, random_state: int = 33):
        """
        get random audio links by feature
        """
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        return audio
        
    
    # play sound
    def play_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        print(f"{key}: {value}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)
    
    
    def play_sample_by_idx(self, idx: int = None, feature: str = 'primary_label'):
        audio = self.meta['full_path'].values[idx]
        print(f"{feature}: {self.meta[feature].values[idx]}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)


    def play_sample(self, file_path):
        return ipd.display(ipd.Audio(file_path)) # ipd.Audio(audio)
    
    
    # get sample, sr
    def load_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33, trim=True):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        y, sr = librosa.load(audio)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr

    
    def load_sample_by_idx(self, idx, trim=True):
        y, sr = librosa.load(self.meta['full_path'].values[idx])
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    def load_sample(self, file_path, trim=True):
        y, sr = librosa.load(file_path)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    # plot waveform
    def waveplot(self, samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        """
        """
        
        if not isinstance(samples, list):
            samples = [samples]
            
        if not isinstance(sample_rates, list):
            sample_rates = [sample_rates]
            
        if names and not isinstance(names, list):
            names = [names]
        
        if len(samples) == 1:
            sample = samples[0]
            sr = sample_rates[0]
            
            fig, ax = plt.subplots(1, figsize = (16, 3))
            fig.suptitle('Sound Waves', fontsize=16)
            librosa.display.waveshow(y=sample, sr=sr, color=self.colors[0], ax=ax)
            
            if names:
                name = names[0]
                ax.set_ylabel(name, fontsize=13)
            
        elif len(samples) > 1:
            n = len(samples)
            fig, ax = plt.subplots(n, figsize = (16, 3*n))
            fig.suptitle('Sound Waves', fontsize=16)
            
            for i in range(len(samples)):
                librosa.display.waveshow(y=samples[i], sr=sample_rates[i], color=self.colors[i % 6], ax=ax[i])
                
                if names:
                    ax[i].set_ylabel(names[i], fontsize=13)
        else:
            print("samples need to contain more than one")
            assert len(samples) >= 1
            
            
    def wave_plot_by_idx(self, idx, trim=True, feature: str = "primary_label"):
        """
        """
        
        y, sr = self.load_sample_by_idx(idx, trim=trim)
            
        fig, ax = plt.subplots(1, figsize = (16, 3))
        fig.suptitle('Sound Waves', fontsize=16)
        librosa.display.waveshow(y=y, sr=sr, color=self.colors[0], ax=ax)
        
        name = self.meta[feature].values[idx]
        ax.set_ylabel(name, fontsize=13)
            
    
    # plot spectrogram
    def spectrogram(self, samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        """
        """
        
        if not isinstance(samples, list):
            samples = [samples]
            
        if not isinstance(sample_rates, list):
            sample_rates = [sample_rates]
            
        if names and not isinstance(names, list):
            names = [names]
            
        if len(samples) == 1:
            # Short-time Fourier transform (STFT)
            stft = np.abs(librosa.stft(samples[0], n_fft=self.n_fft, hop_length=self.hop_length))
            # Convert an amplitude spectrogram to Decibels-scaled spectrogram.
            DB_spec = librosa.amplitude_to_db(stft, ref = np.max)
            
            # === PLOT ===
            fig, ax = plt.subplots(1, 1, figsize=(12, 6))
            fig.suptitle('Log Frequency Spectrogram', fontsize=16)
            # fig.delaxes(ax[1, 2])
            
            img=librosa.display.specshow(DB_spec, sr=sample_rates[0], hop_length=self.hop_length, x_axis = 'time',
                         y_axis = 'log', cmap = 'cool', ax=ax)
            ax.set_title(names[0], fontsize=13) 
            plt.colorbar(img,ax=ax)
            
        elif len(samples) > 1:
            n = len(samples)
            fig, ax = plt.subplots(n, figsize = (16, 6*n))
            fig.suptitle('Log Frequency Spectrogram', fontsize=16)
            
            for i in range(len(samples)):
                stft = np.abs(librosa.stft(samples[i], n_fft=self.n_fft, hop_length=self.hop_length))
                # Convert an amplitude spectrogram to Decibels-scaled spectrogram.
                DB_spec = librosa.amplitude_to_db(stft, ref = np.max)
                
                img=librosa.display.specshow(DB_spec, sr=sample_rates[i], hop_length=self.hop_length, x_axis = 'time',
                         y_axis = 'log', cmap = 'cool', ax=ax[i])
                
                if names:
                    ax[i].set_title(names[i], fontsize=13)
            
        
        else:
            print("samples need to contain more than one")
            assert len(samples) >= 1
            
        
    def plot_rmse(self, samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        S, phase = librosa.magphase(librosa.stft(samples))
        S_db=librosa.amplitude_to_db(S, ref=np.max)
        rms = librosa.feature.rms(S=S)
        fig, ax = plt.subplots(nrows=2, sharex=True,figsize = (16, 6))
        times = librosa.times_like(rms)
        ax[0].semilogy(times, rms[0], label='RMS Energy')
        ax[0].set(xticks=[])
        ax[0].legend()
        ax[0].label_outer()
        librosa.display.specshow(S_db,
                                 y_axis='log', x_axis='time', ax=ax[1])
        ax[1].set(title='log Power spectrogram')
        plt.show()
        

In [ ]:
class AudioLibrosa():
    def __init__(self, n_fft, hop_length):
        self.root_dir = ROOTDIR
        # edit metadata
        self.meta = pd.read_csv(fr"{self.root_dir}/train_metadata.csv")
        self.meta["full_path"] = self.root_dir + '/' + 'train_audio' + '/' + self.meta['filename']
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.colors = ["#009DFF", "#A300F9", "#4300FF", "#00FFB0", "#D9FF00", "r"]
    
    # get random sample by feature
    def get_random_sample_by_feature(self, key: str = 'primary', value: str = None, random_state: int = 33):
        """
        get random audio links by feature
        """
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        return audio
        
    
    # play sound
    def play_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        print(f"{key}: {value}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)
    
    
    def play_sample_by_idx(self, idx: int = None, feature: str = 'primary_label'):
        audio = self.meta['full_path'].values[idx]
        print(f"{feature}: {self.meta[feature].values[idx]}")
        return ipd.display(ipd.Audio(audio)) # ipd.Audio(audio)


    def play_sample(self, file_path):
        return ipd.display(ipd.Audio(file_path)) # ipd.Audio(audio)
    
    
    # get sample, sr
    def load_random_sample_by_feature(self, key: str = None, value: str = None, random_state: int = 33, trim=True):
        audio = self.meta[self.meta[key] == value].sample(1, random_state = 33)['full_path'].values[0]
        y, sr = librosa.load(audio)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr

    
    def load_sample_by_idx(self, idx, trim=True):
        y, sr = librosa.load(self.meta['full_path'].values[idx])
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    def load_sample(self, file_path, trim=True):
        y, sr = librosa.load(file_path)
        
        if trim:
            y, _ = librosa.effects.trim(y)
            
        return y, sr
    
    
    # plot waveform
    def waveplot(self, samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        """
        """
        
        if not isinstance(samples, list):
            samples = [samples]
            
        if not isinstance(sample_rates, list):
            sample_rates = [sample_rates]
            
        if names and not isinstance(names, list):
            names = [names]
        
        if len(samples) == 1:
            sample = samples[0]
            sr = sample_rates[0]
            
            fig, ax = plt.subplots(1, figsize = (16, 3))
            fig.suptitle('Sound Waves', fontsize=16)
            librosa.display.waveshow(y=sample, sr=sr, color=self.colors[0], ax=ax)
            
            if names:
                name = names[0]
                ax.set_ylabel(name, fontsize=13)
            
        elif len(samples) > 1:
            n = len(samples)
            fig, ax = plt.subplots(n, figsize = (16, 3*n))
            fig.suptitle('Sound Waves', fontsize=16)
            
            for i in range(len(samples)):
                librosa.display.waveshow(y=samples[i], sr=sample_rates[i], color=self.colors[i % 6], ax=ax[i])
                
                if names:
                    ax[i].set_ylabel(names[i], fontsize=13)
        else:
            print("samples need to contain more than one")
            assert len(samples) >= 1
            
            
    def wave_plot_by_idx(self, idx, trim=True, feature: str = "primary_label"):
        """
        """
        
        y, sr = self.load_sample_by_idx(idx, trim=trim)
            
        fig, ax = plt.subplots(1, figsize = (16, 3))
        fig.suptitle('Sound Waves', fontsize=16)
        librosa.display.waveshow(y=y, sr=sr, color=self.colors[0], ax=ax)
        
        name = self.meta[feature].values[idx]
        ax.set_ylabel(name, fontsize=13)
            
    
    # plot spectrogram
    def spectrogram(self, samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        """
        """
        
        if not isinstance(samples, list):
            samples = [samples]
            
        if not isinstance(sample_rates, list):
            sample_rates = [sample_rates]
            
        if names and not isinstance(names, list):
            names = [names]
            
        if len(samples) == 1:
            # Short-time Fourier transform (STFT)
            stft = np.abs(librosa.stft(samples[0], n_fft=self.n_fft, hop_length=self.hop_length))
            # Convert an amplitude spectrogram to Decibels-scaled spectrogram.
            DB_spec = librosa.amplitude_to_db(stft, ref = np.max)
            
            # === PLOT ===
            fig, ax = plt.subplots(1, 1, figsize=(12, 6))
            fig.suptitle('Log Frequency Spectrogram', fontsize=16)
            # fig.delaxes(ax[1, 2])
            
            img=librosa.display.specshow(DB_spec, sr=sample_rates[0], hop_length=self.hop_length, x_axis = 'time',
                         y_axis = 'log', cmap = 'cool', ax=ax)
            ax.set_title(names[0], fontsize=13) 
            plt.colorbar(img,ax=ax)
            
        elif len(samples) > 1:
            n = len(samples)
            fig, ax = plt.subplots(n, figsize = (16, 6*n))
            fig.suptitle('Log Frequency Spectrogram', fontsize=16)
            
            for i in range(len(samples)):
                stft = np.abs(librosa.stft(samples[i], n_fft=self.n_fft, hop_length=self.hop_length))
                # Convert an amplitude spectrogram to Decibels-scaled spectrogram.
                DB_spec = librosa.amplitude_to_db(stft, ref = np.max)
                
                img=librosa.display.specshow(DB_spec, sr=sample_rates[i], hop_length=self.hop_length, x_axis = 'time',
                         y_axis = 'log', cmap = 'cool', ax=ax[i])
                
                if names:
                    ax[i].set_title(names[i], fontsize=13)
            
        
        else:
            print("samples need to contain more than one")
            assert len(samples) >= 1
            
        
    def plot_rmse(self, samples: List[np.ndarray], sample_rates: List[str], names: List[str] = None):
        S, phase = librosa.magphase(librosa.stft(samples))
        S_db=librosa.amplitude_to_db(S, ref=np.max)
        rms = librosa.feature.rms(S=S)
        fig, ax = plt.subplots(nrows=2, sharex=True,figsize = (16, 6))
        times = librosa.times_like(rms)
        ax[0].semilogy(times, rms[0], label='RMS Energy')
        ax[0].set(xticks=[])
        ax[0].legend()
        ax[0].label_outer()
        librosa.display.specshow(S_db,
                                 y_axis='log', x_axis='time', ax=ax[1])
        ax[1].set(title='log Power spectrogram')
        plt.show()
        
    
test_d = AudioLibrosa(n_fft=Config.n_fft, hop_length=Config.hop_length)
birds = list(train_df.primary_label.unique()[0:6])

In [ ]:
test_d.spectrogram(trimed_samples, sample_rates, birds)